In [1]:
import pandas as pd 
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from lightgbm import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import re
from collections import Counter
%matplotlib inline

In [2]:
train = pd.read_csv('train.csv')
#service=pd.read_csv("input/service_use_history.csv")
train = train.set_index('user_id')
train.head()

,user_agent,entry_date,open_date,birthday,id_status,gender,blood_type,pay_method_type,course_name,price_type,entry_from,privilege_name,prefecture,close_flag
user_id,,,,,,,,,,,,,,
YKJGBC19356,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_9_4...,2016-12-29,2017-01-26,1964-04-10,新規,男性,A,クレジットカード,@nifty光,マンション2年プラン,代理店取次,キャッシュバック還元,鹿児島,0
JMFEBB08712,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,2018-04-21,2018-05-29,1980-11-24,新規,女性,O,クレジットカード,@nifty光,ホーム2年プラン,代理店取次,付帯サービス無料,北海道,0
EXLFTB26665,Mozilla/5.0 (iPhone; CPU iPhone OS 3_1_3 like ...,2015-11-28,2016-04-05,1972-08-01,既存,女性,AB,クレジットカード,@nifty auひかり,ホーム2年プラン,ノジマ取次,月額割引還元,長野,0
MWEEMY19722,Opera/8.10.(X11; Linux i686; nn-NO) Presto/2.9...,2019-07-16,2019-08-15,1966-03-15,新規,男性,B,クレジットカード,@nifty光,マンション2年プラン,代理店取次,キャッシュバック還元,福島,0
WWTLYT31226,Mozilla/5.0 (iPad; CPU iPad OS 4_2_1 like Mac ...,2019-09-12,2020-01-06,1988-04-29,新規,男性,A,クレジットカード,@nifty auひかり,マンション3年プラン,WEB代理店,付帯サービス無料,愛知,0


In [3]:
os=train["user_agent"][0].split("/")
os_name=[]
for i in range(train.shape[0]):
    os=train["user_agent"][i].split("/")
    if os[0] == "Mozilla":
        os_name.append(1)
    else:
        os_name.append(0)
train["os_name"]=os_name
train.head()

,user_agent,entry_date,open_date,birthday,id_status,gender,blood_type,pay_method_type,course_name,price_type,entry_from,privilege_name,prefecture,close_flag,os_name
user_id,,,,,,,,,,,,,,,
YKJGBC19356,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_9_4...,2016-12-29,2017-01-26,1964-04-10,新規,男性,A,クレジットカード,@nifty光,マンション2年プラン,代理店取次,キャッシュバック還元,鹿児島,0,1
JMFEBB08712,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,2018-04-21,2018-05-29,1980-11-24,新規,女性,O,クレジットカード,@nifty光,ホーム2年プラン,代理店取次,付帯サービス無料,北海道,0,1
EXLFTB26665,Mozilla/5.0 (iPhone; CPU iPhone OS 3_1_3 like ...,2015-11-28,2016-04-05,1972-08-01,既存,女性,AB,クレジットカード,@nifty auひかり,ホーム2年プラン,ノジマ取次,月額割引還元,長野,0,1
MWEEMY19722,Opera/8.10.(X11; Linux i686; nn-NO) Presto/2.9...,2019-07-16,2019-08-15,1966-03-15,新規,男性,B,クレジットカード,@nifty光,マンション2年プラン,代理店取次,キャッシュバック還元,福島,0,0
WWTLYT31226,Mozilla/5.0 (iPad; CPU iPad OS 4_2_1 like Mac ...,2019-09-12,2020-01-06,1988-04-29,新規,男性,A,クレジットカード,@nifty auひかり,マンション3年プラン,WEB代理店,付帯サービス無料,愛知,0,1


In [ ]:
count = [0]*6
train['device'] = np.zeros(len(train["user_agent"]), dtype = int)
for i in range(train['user_agent'].shape[0]):
    sam = str(train['user_agent'][i])
    agent_flag = 0
    if 'Windows' in sam:
        count[0]+=1
        agent_flag = 1
    elif 'Android' in sam:
        count[1]+=1
        agent_flag = 2
    elif 'iPad' in sam:
        count[2]+=1
        agent_flag = 3
    elif 'Linux' in sam:
        count[3]+=1
        agent_flag = 4
    elif 'Mac' in sam:
        count[4]+=1
        agent_flag = 5
    elif 'iPhone' in sam:
        count[5]+=1
        agent_flag = 5
    else:
        agent_flag = 0
    train['device'][i] = agent_flag

train.head()

C:\Users\riku\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
datetime_columns = ['user_agent', 'entry_date', 'open_date',"blood_type"]
for column in datetime_columns:
    train = train.drop(column, axis=1)

In [ ]:
train.head()

In [ ]:
urban=["東京","神奈川","大阪","愛知","埼玉","千葉","兵庫","北海道","福岡"]
for i in range(train['prefecture'].shape[0]):
    if (train['prefecture'][i] in urban) == True:
        train['arban'] = 1
    elif (train['prefecture'][i] in urban) == False:
        train['arban'] = 0

In [ ]:
#object型➡datetime64型の変換
train["X"]=pd.to_datetime(train["birthday"],format="%Y-%m-%d")

#年齢のcolumnを追加
train["age"]=2020-train["X"].dt.year

train=train[train["age"]>=0]
#標準化
#train["age"]=scipy.stats.zscore(train["age"])


In [ ]:
category_columns = ['id_status', 'gender',"pay_method_type",'course_name',"entry_from" ,'price_type', 'privilege_name']
encoders = {}
for column in category_columns:
    le = LabelEncoder()
    le.fit(train[column])
    train[column] = le.transform(train[column])
    encoders[column] = le    

In [ ]:
datetime_columns = ['birthday',"prefecture","X"]
for column in datetime_columns:
    train = train.drop(column, axis=1)

In [ ]:
y = train['close_flag']
X = train.drop('close_flag', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)

y_pred_prob = rf.predict_proba(X_test)[:,1]
y_pred = [1 if prob > 0.5 else 0 for prob in y_pred_prob]

print(f'accuracy_score: {accuracy_score(y_test, y_pred)}')
print(f'roc_auc_score: {roc_auc_score(y_test, y_pred_prob)}')

pd.DataFrame(confusion_matrix(y_test, y_pred), index=['actual_0', 'actual_1'], columns=['predicted_0', 'predicted_1'])

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
xgb_params = {
        # 評価指標
        'eval_metric': 'logloss',
    }

bst = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=100,  
                    )
    
y_pred_proba = bst.predict(dtest)
    
y_pred = np.where(y_pred_proba > 0.5, 1, 0)
    
acc = accuracy_score(y_test, y_pred)
print('Accuracy:', acc)
print(f'roc_auc_score: {roc_auc_score(y_test, y_pred_proba)}')
pd.DataFrame(confusion_matrix(y_test, y_pred), index=['actual_0', 'actual_1'], columns=['predicted_0', 'predicted_1'])

In [ ]:
import xgboost as xgb
from lightgbm import Dataset
import pandas as pd 
import numpy as np
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from lightgbm import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import re
from collections import Counter
%matplotlib inline

In [ ]:
lgb_params = {"objective":"binary", 
              "metric":"binary_logloss", 
              "verbosity": -1}

lgb_train = lgb.Dataset(
                 X_train, 
                 y_train)

lgb = lgb.train(lgb_params, lgb_train)

y_pred_proba=lgb.predict(X_test)
print(f'roc_auc_score: {roc_auc_score(y_test, y_pred_proba)}')

In [ ]:
test = pd.read_csv("input/test.csv")
test = test.set_index("user_id")

os=test["user_agent"][0].split("/")
os_name=[]
for i in range(test.shape[0]):
    os=test["user_agent"][i].split("/")
    if os[0] == "Mozilla":
        os_name.append(1)
    else:
        os_name.append(0)
test["os_name"]=os_name


count = [0]*6
test['device'] = np.zeros(len(test["user_agent"]), dtype = int)
for i in range(test['user_agent'].shape[0]):
    sam = str(test['user_agent'][i])
    agent_flag = 0
    if 'Windows' in sam:
        count[0]+=1
        agent_flag = 1
    elif 'Android' in sam:
        count[1]+=1
        agent_flag = 2
    elif 'iPad' in sam:
        count[2]+=1
        agent_flag = 3
    elif 'Linux' in sam:
        count[3]+=1
        agent_flag = 4
    elif 'Mac' in sam:
        count[4]+=1
        agent_flag = 5
    elif 'iPhone' in sam:
        count[5]+=1
        agent_flag = 5
    else:
        agent_flag = 0
    test['device'][i] = agent_flag



datetime_columns = ['user_agent', 'entry_date', 'open_date',"blood_type"]
for column in datetime_columns:
    test = test.drop(column, axis=1)

urban=["東京","神奈川","大阪","愛知","埼玉","千葉","兵庫","北海道","福岡"]
for i in range(test['prefecture'].shape[0]):
    if (test['prefecture'][i] in urban) == True:
        test['arban'] = 1
    elif (test['prefecture'][i] in urban) == False:
        test['arban'] = 0
    
#object型➡datetime64型の変換
test["X"]=pd.to_datetime(test["birthday"],format="%Y-%m-%d")

#年齢のcolumnを追加
test["age"]=2020-test["X"].dt.year


category_columns = ['id_status', 'gender',"pay_method_type","entry_from",'course_name', 'price_type', 'privilege_name']
encoders = {}
for column in category_columns:
    le = LabelEncoder()
    le.fit(test[column])
    test[column] = le.transform(test[column])
    encoders[column] = le       

datetime_columns = ['birthday',"prefecture","X"]
for column in datetime_columns:
    test = test.drop(column, axis=1)
    
    
    
    
test["age"].fillna(test.age.mean(), inplace=True)
print(test.isnull().sum())
test.head()

In [ ]:
submission = pd.DataFrame({'user_id': test.index, 'close_flag':  lgb.predict(test)})
submission.head()
submission.to_csv('output/my_submission1.csv', index=False)
! head output/my_submission.csv